### Fix NNCAM output file
See: https://towardsdatascience.com/how-to-create-xarray-datasets-cf1859c95921

In [1]:
import pdb
import xarray as xr
import numpy as np

**norm & data files**

In [24]:
dataPath = '/work/bd1179/b309172/data/Aqua_0K_Raw_Output_NNCAM'
tmpPath  = '/work/bd1179/b309172/data/Aqua_0K_Raw_Output'

# Open datasets
data_fn = 'Final_SPCAM3_DNN_Year.nc'
data_ds = xr.open_dataset(dataPath+'/'+data_fn)

# Template data
tmp_fn = 'AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-01-01-0000-01-31.nc'
tmp_ds = xr.open_dataset(tmpPath+'/'+tmp_fn)

In [26]:
tmp_ds

<xarray.Dataset>
Dimensions:    (ilev: 31, lat: 64, lev: 30, lon: 128, tbnd: 2, time: 1488)
Coordinates:
  * ilev       (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * lev        (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * time       (time) object 0000-01-01 00:00:00 ... 0000-01-31 23:30:00
Dimensions without coordinates: tbnd
Data variables: (12/16)
    FLNS       (time, lat, lon) float32 ...
    FLNT       (time, lat, lon) float32 ...
    FSNS       (time, lat, lon) float32 ...
    FSNT       (time, lat, lon) float32 ...
    P0         float64 ...
    PHQ        (time, lev, lat, lon) float32 ...
    ...         ...
    TPHYSTND   (time, lev, lat, lon) float32 ...
    hyai       (ilev) float64 ...
    hyam       (lev) float64 ...
    hybi       (ilev) float64 ...
    hybm       (lev) float64 ...
    time_bnds  (time, tbnd) object ...
Attributes:
    Conventions:  CF-1.0
    source:       CAM
    case:         AndKua_aqua_SPCAM3.0_sp_fbp_f4
    title:        
    logname:      tg847872
    host:         
    Version:      $Name:  $
    revision_Id:  $Id: history.F90,v 1.26.2.38 2003/12/15 18:52:35 hender Exp $
    history:      Mon Nov  8 14:53:07 2021: ncks -v TPHYSTND,PHQ,TAP,QAP,PREC...
    NCO:          netCDF Operators version 4.9.3 (Homepage = http://nco.sf.ne...

In [3]:
data_ds

<xarray.Dataset>
Dimensions:     (lev: 65, sample: 12181504)
Coordinates:
  * sample      (sample) int64 0 1 2 3 4 ... 12181500 12181501 12181502 12181503
  * lev         (lev) int64 0 1 2 3 4 5 6 7 8 9 ... 56 57 58 59 60 61 62 63 64
Data variables:
    Prediction  (sample, lev) float64 ...

In [113]:
tmp_ds['lev']

<xarray.DataArray 'lev' (lev: 30)>
array([  3.643466,   7.59482 ,  14.356632,  24.61222 ,  38.2683  ,  54.59548 ,
        72.012451,  87.82123 , 103.317127, 121.547241, 142.994039, 168.22508 ,
       197.908087, 232.828619, 273.910817, 322.241902, 379.100904, 445.992574,
       524.687175, 609.778695, 691.38943 , 763.404481, 820.858369, 859.534767,
       887.020249, 912.644547, 936.198398, 957.48548 , 976.325407, 992.556095])
Coordinates:
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
Attributes:
    long_name:      hybrid level at midpoints (1000*(A+B))
    units:          level
    positive:       down
    standard_name:  atmosphere_hybrid_sigma_pressure_coordinate
    formula_terms:  a: hyam b: hybm p0: P0 ps: PS

In [70]:
prect = np.array(data_ds['Prediction'][:,64]).reshape((len(tmp_ds['time'][:-1]),len(tmp_ds['lat']),len(tmp_ds['lon'])))

In [73]:
phq = np.array(data_ds['Prediction'][:,0:30])
print(phq.shape)

(12181504, 30)


In [74]:
phq = np.array(data_ds['Prediction'][:,0:30]).reshape((len(tmp_ds['time'][:-1]), len(tmp_ds['lev']),len(tmp_ds['lat']),len(tmp_ds['lon'])))

In [75]:
phq.shape

(1487, 30, 64, 128)

**3D variables**

In [ ]:
# phq      = np.array(data_ds['Prediction'][:,0:30]).reshape((len(tmp_ds['time'][:-1]), len(tmp_ds['lev']),len(tmp_ds['lat']),len(tmp_ds['lon'])))
# tphystnd = np.array(data_ds['Prediction'][:,30:60]).reshape((len(tmp_ds['time'][:-1]), len(tmp_ds['lev']),len(tmp_ds['lat']),len(tmp_ds['lon'])))

In [108]:
phq      = np.zeros([len(tmp_ds['time'][:-1]), len(tmp_ds['lev']),len(tmp_ds['lat']),len(tmp_ds['lon'])])
tphystnd = np.zeros([len(tmp_ds['time'][:-1]), len(tmp_ds['lev']),len(tmp_ds['lat']),len(tmp_ds['lon'])])
count = 0
for iLev in range(len(tmp_ds['lev'])):
    phq[:,count,:,:]      = np.array(data_ds['Prediction'][:,count+30]).reshape((len(tmp_ds['time'][:-1]),len(tmp_ds['lat']),len(tmp_ds['lon'])))
    tphystnd[:,count,:,:] = np.array(data_ds['Prediction'][:,count]).reshape((len(tmp_ds['time'][:-1]),len(tmp_ds['lat']),len(tmp_ds['lon'])))
    count += 1

**2D variables**

In [109]:
fsnt     = np.array(data_ds['Prediction'][:,60]).reshape((len(tmp_ds['time'][:-1]),len(tmp_ds['lat']),len(tmp_ds['lon'])))
fsns     = np.array(data_ds['Prediction'][:,61]).reshape((len(tmp_ds['time'][:-1]),len(tmp_ds['lat']),len(tmp_ds['lon'])))
flnt     = np.array(data_ds['Prediction'][:,62]).reshape((len(tmp_ds['time'][:-1]),len(tmp_ds['lat']),len(tmp_ds['lon'])))
flns     = np.array(data_ds['Prediction'][:,63]).reshape((len(tmp_ds['time'][:-1]),len(tmp_ds['lat']),len(tmp_ds['lon'])))
prect    = np.array(data_ds['Prediction'][:,64]).reshape((len(tmp_ds['time'][:-1]),len(tmp_ds['lat']),len(tmp_ds['lon'])))

**Create new file**

In [116]:
# create coordinates
lons = tmp_ds['lon']
lats = tmp_ds['lat']
levs = tmp_ds['lev']
time = tmp_ds['time'][:-1]
lon_meshgrid, lat_meshgrid = np.meshgrid(lons, lats, indexing='ij')

In [117]:
phq.shape

(1487, 30, 64, 128)

In [121]:
ds = xr.Dataset(
    data_vars=dict(
        phq=(['time','lev','lat','lon'], phq, {"units":"kg/kg/s", "long_name":"Specific humidity physics tendency"}),
        tphystnd=(['time','lev','lat','lon'], tphystnd, {"units":"K/s", "long_name":"T physics tendency"}),
        fsnt=(['time','lat','lon'], fsnt, {"units":"W/m2", "long_name":"Net solar flux at top of model"}),
        fsns=(['time','lat','lon'], fsns, {"units":"W/m2", "long_name":"Net solar flux at surface"}),
        flnt=(['time','lat','lon'], flnt, {"units":"W/m2", "long_name":"Net longwave flux at top of model"}),
        flns=(['time','lat','lon'], flns, {"units":"W/m2", "long_name":"Net longwave flux at surface"}),
        prect=(['time','lat','lon'], prect, {"units":"m/s", "long_name":"Total (convective and large-scale) precipitation rate"})
    ),
    coords={
        'time':(["time"], time),
        'lev':(["lev"], levs),
        'lat':(["lat"], lats),
        'lon':(["lon"], lons),
    },
    attrs={'description':"Reconstructed: Final_SPCAM3_DNN_Year.nc",
                'author':'Fernando Iglesias-Suarez', 
                 'email':'fernando.iglesias-suarez@dlr.de'},
)

In [122]:
ds.to_netcdf('AndKua_aqua_NNCAM3.0_sp_fbp_f4.cam2.h1.0000-01-01-0000-01-31.nc')